# Run for setup

## Drive connection

In [24]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

!ls '/content/drive/My Drive/SonART'

Mounted at /content/drive
Code  labels_config.json  saved_model  subgroups_label.json


## Import

In [2]:
%pip install --quiet --upgrade diffusers transformers accelerate mediapy peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.9 MB/s eta 0:00:00


In [3]:
!pip install torchlibrosa

In [4]:
pip install pydub

In [5]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import librosa
import torch
import datetime
import time
from PIL import Image
import shutil
import threading
from multiprocessing import Process, Manager
from IPython.display import Audio
from pydub import AudioSegment, effects
from pydub.effects import compress_dynamic_range
import ipywidgets as widgets
from google.colab import output, files
import json

#collegamento con processing

import socket
import cv2
import base64
import time

In [6]:
path = "/content/drive/MyDrive/SonART/Code"
if os.path.exists(path):
    print("La cartella esiste!")
else:
    print("Errore: La cartella non è stata trovata. Controlla il percorso.")

La cartella esiste!


In [7]:
sys.path.append("/content/drive/MyDrive/SonART/Code")

In [8]:
from panns_inference import AudioTagging, SoundEventDetection, labels

In [9]:
category_path = "/content/drive/MyDrive/SonART/subgroups_label.json"

with open(category_path, "r") as f:
    category_mapping = json.load(f)

In [29]:
change_labels_path = "/content/drive/MyDrive/SonART/labels_config.json"

with open(change_labels_path, "r") as f:
    labels_mapping = json.load(f)

replacements = labels_mapping["replacements"]
groups = labels_mapping["groups"]

## Processing connection

In [10]:
# Dati del server
host = '4.tcp.eu.ngrok.io'
port = 10746

## Retrieve Audio model

In [11]:
# Percorso del modello su Drive
checkpoint_model = "/content/drive/My Drive/SonART/saved_model/panns/panns_model.pth"

# Carica il modello salvato
model = torch.load(checkpoint_model, map_location="cuda" if torch.cuda.is_available() else "cpu", weights_only=False)

## Download image model

In [12]:
import torch
from diffusers import LCMScheduler, AutoPipelineForText2Image
import mediapy as media
import random
import sys
import torch

model_id = "stabilityai/stable-diffusion-xl-base-1.0"
adapter_id = "latent-consistency/lcm-lora-sdxl"

pipe = AutoPipelineForText2Image.from_pretrained(model_id, torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

# load and fuse lcm lora
pipe.load_lora_weights(adapter_id)
pipe.fuse_lora()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

...

vae/diffusion_pytorch_model.fp16.safeten(…):   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

The config attributes {'skip_prk_steps': True} were passed to LCMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


pytorch_lora_weights.safetensors:   0%|          | 0.00/394M [00:00<?, ?B/s]

No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new
No LoRA keys associated to CLIPTextModelWithProjection found with the prefix='text_encoder_2'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModelWithProjection related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


## Functions and classes needed for audio + image model

In [13]:
def best_tag(clipwise_output):
    """Visualization of audio tagging result.

    Args:
      clipwise_output: (classes_num,)
    """
    sorted_indexes = np.argsort(clipwise_output)[::-1]

    return sorted_indexes[0]

In [14]:
class ObservableList(list):
    def __init__(self, *args, save_dir="/content/images"):
        super().__init__(*args)
        self.save_dir = save_dir
        self.saved_images = {}  # Dizionario per tenere traccia dei file salvati
        os.makedirs(self.save_dir, exist_ok=True)  # Creazione della cartella se non esiste
        self.used_timestamps = []

    def append(self, item, i, background, style, time_stamp):
        super().append(item + str(i))  # Chiamata originale
        self.on_append(item, i, background, style, time_stamp)

    def on_append(self, item, i, background, style, time_stamp):
        prompt = item
        if background is not None:
            prompt += f" with {background} as background"
        if style is not None:
            prompt += f" in {style} style"

        image = pipe(prompt=prompt, num_inference_steps=5, guidance_scale=0).images[0]

        millis_stamp = int(time_stamp * 1000)
        filename = f"{self.save_dir}/{millis_stamp}.png"

        # Salva l'immagine
        image.save(filename)
        self.saved_images[item + str(i)] = filename  # Associa l'elemento al percorso dell'immagine

        # Salva timestamps
        self.used_timestamps.append(time_stamp)

    def get_saved_image(self, item):
        """Ritorna il percorso dell'immagine salvata associata all'elemento"""
        return self.saved_images.get(item, None)

    def clear_saved_images(self):
        """Cancella tutte le immagini salvate nella cartella e svuota il dizionario"""
        for filename in os.listdir(self.save_dir):
              if filename.endswith(".png"):
                  os.remove(os.path.join(self.save_dir, filename))

        self.saved_images.clear()  # Svuota il dizionario


**Se non avete già un modello nel percorso allora passate model=None**

In [15]:
checkpoint_path = '/content/drive/My Drive/SonART/saved_model/panns/Cnn14_mAP=0.431.pth' #percorso pesi
checkpoint_model = '/content/drive/My Drive/SonART/saved_model/panns/panns_model.pth'

device = 'cuda' # 'cuda' | 'cpu'
at = AudioTagging(model=model, checkpoint_path=checkpoint_path, checkpoint_model=checkpoint_model, device=device)

Checkpoint path: /content/drive/My Drive/SonART/saved_model/panns/Cnn14_mAP=0.431.pth
✅ La cartella esiste già: /content/drive/My Drive/SonART/saved_model/panns
✅ Il file esiste già, nessun download necessario.
Il modello esiste già!
GPU number: 1


In [16]:
def generate_and_play_audio_with_images(audio_path, tag_list):

  display(Audio(audio_path, autoplay=True))

  for i, element in enumerate(tag_list):
      image_path = tag_list.get_saved_image(element)
      image = cv2.imread(image_path)

      timestamp = os.path.splitext(os.path.basename(image_path))[0]

      # Codifica immagine
      _, img_encoded = cv2.imencode('.jpg', image)
      img_base64 = base64.b64encode(img_encoded.tobytes()).decode('utf-8')

      try:
        client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client.connect((host, port))

        client.sendall((f"{timestamp}\n").encode('utf-8'))         # prima riga: filename
        client.sendall((f"{img_base64}\n").encode('utf-8'))        # seconda riga: base64

        print(f"📤 Inviata immagine con timestamp {timestamp}")
        client.close()

      except Exception as e:
          print(f"⚠️ Errore durante la connessione o l'invio: {e}")

In [32]:
def replace_if_trouble_label(input_string):
    if input_string in replacements:
        return replacements[input_string]

    for group, values in groups.items():
        if input_string in values:
            return group

    return input_string

In [18]:
def elaborate(audio_path, background, style, selected_categories):

    audio, sr = librosa.load(audio_path, sr=32000, mono=True)


    # **Specifica la durata dei segmenti in secondi**
    segment_duration = 2 # Durata del segmento in secondi (esempio: 2 secondi)
    hop_duration = 0.1  # Durata del salto tra segmenti in secondi (esempio: 1 secondo)
    #time_stamps = np.arange(0, len(audio)/sr, hop_duration)

    # Calcolare la lunghezza di ogni segmento in campioni
    segment_length = int(segment_duration * 32000)
    hop_length = int(hop_duration * 32000)

    # Dividere il waveform in segmenti con il salto specificato
    segments = []
    for start in range(0, len(audio) - segment_length + 1, hop_length):
        segment = audio[start:start + segment_length]
        segments.append(segment)

    # Aggiungere l'ultimo segmento se non coperto dal ciclo (padding se necessario)
    if len(audio) % hop_length != 0:
        last_segment = audio[-segment_length:]
        segments.append(last_segment)

    # Trasformare i segmenti in un array numpy
    segments = np.stack(segments)

    time_stamps = np.arange(len(segments)) * hop_duration

    # Mostrare dettagli
    print(f"Numero di segmenti: {len(segments)}, Forma di ogni segmento: {segments[0].shape}")

    '''
    if "complete" not in selected_categories:
        temp_labels = []
        for categories in selected_categories:
            if categories in category_mapping:
                temp_labels.extend(category_mapping[categories])
        allowed_labels = set(temp_labels)
    else:
        allowed_labels = None
        '''

    # Variabile per tracciare la parola precedentemente stampata
    last_printed_word = None

    # Label escluse
    excluded_labels = ["Animal",
    "Wild animals",
    "Domestic animals, pets",
    "Music",
    "Musical instrument",
    "Plucked string instrument",
    "Sound effect",
    "Siren",
    "Vehicle",
    "Engine",
    "Silence"]

    tag_list = ObservableList()
    last_event = None

    for i, segment in enumerate(segments):
        segment = segment[None, :]  # (batch_size, segment_samples)

        (clipwise_output, embedding) = at.inference(segment)
        """clipwise_output: (batch_size, classes_num), embedding: (batch_size, embedding_size)"""

        '''
        if allowed_labels is not None:
            labels_str = [str(label) for label in labels]
            allowed_str = [str(label) for label in allowed_labels]
            mask = np.isin(labels_str, allowed_str)
            clipwise_output[0][~mask] = 0
        '''

        # Filtra le etichette escluse
        if  excluded_labels is not None:
            labels_str = [str(label) for label in labels]
            mask = np.isin(labels_str, excluded_labels)
            clipwise_output[0][mask] = 0

        index = best_tag(clipwise_output[0])
        predicted_class = np.array(labels)[index]

        # Cambio label per migliorare generazione immagine
        predicted_class = replace_if_trouble_label(predicted_class)

        clipwise_output = clipwise_output[0]

        # Stampa se non è uguale all'ultima stampata e supera la soglia
        if predicted_class != last_printed_word:
            if clipwise_output[index] >= 0.400:
                print('{}: {:.3f}'.format(predicted_class,
                clipwise_output[index]))
                print(str(datetime.timedelta(seconds=time_stamps[i])))
                tag_list.append(predicted_class, i, background, style, time_stamps[i])
                last_printed_word = predicted_class  # Aggiorna l'ultima parola stampata
                last_event = i

    return tag_list


In [19]:
#tag_list.clear_saved_images()

# GUI

In [23]:
# ──────────── GUI Aggiornata ────────────
text1 = widgets.Text(value='nature', placeholder='Es. snowy forest', description='🎨 Background:', layout=widgets.Layout(width='70%'))
text2 = widgets.Text(value='fantasy art', placeholder='Es. watercolor style', description='🖌️ Style:', layout=widgets.Layout(width='70%'))

categorie = [
    "complete", "Audio Processing Effects Sounds", "Acoustic Environment and Space",
    "Synthetic and Abstract Sounds", "Miscellaneous Sounds and Noises", "Physical Actions Sounds",
    "Motion and Impact Sounds", "Material and Liquid Sounds", "Explosions Sounds",
    "Appliances and Tools", "Alarms and Mechanical Sounds", "Household and Everyday Sounds",
    "Vehicles and Engine Sounds", "Natural Elements and Weather Sounds", "Music",
    "Animals", "People"
]
checkboxes = [widgets.Checkbox(value=False, description=label) for label in categorie]
checkbox_box = widgets.VBox(checkboxes)

# Widget per visualizzare il file caricato
file_info = widgets.HTML(value="<div style='padding:5px; color:gray;'>Nessun file caricato</div>")
upload_button = widgets.Button(description='📤 Carica File', button_style='primary', layout=widgets.Layout(width='50%'))
process_button = widgets.Button(description='🎬 Avvia Elaborazione', button_style='success', layout=widgets.Layout(width='50%'), disabled=True)

status_text = widgets.HTML(value="<div style='padding: 5px;'>Pronto per l'upload</div>")

# Variabile globale per il percorso del file
uploaded_file_path = None
tag_list = None  # Aggiunto per gestire la lista di tag globalmente

# ──────────── Funzione per il caricamento ────────────
def upload_handler(_):
    global uploaded_file_path
    try:
        uploaded = files.upload()
        if not uploaded:
            file_info.value = "<div style='color:red; padding:5px;'>Nessun file selezionato</div>"
            process_button.disabled = True
            return

        # Prendi il primo file caricato
        filename = next(iter(uploaded.keys()))
        content = uploaded[filename]

        # Salva il file temporaneamente
        uploaded_file_path = f"/content/{filename}"
        with open(uploaded_file_path, 'wb') as f:
            f.write(content)

        file_info.value = f"<div style='color:green; padding:5px;'>File caricato: <b>{filename}</b></div>"
        process_button.disabled = False
        status_text.value = "<div style='color:green; padding:5px;'>File pronto per l'elaborazione</div>"

    except Exception as e:
        file_info.value = f"<div style='color:red; padding:5px;'>Errore: {str(e)}</div>"
        process_button.disabled = True

# ──────────── Pulsante Play ────────────
play_button = widgets.Button(description='▶️ Play Audio with Images',
                           button_style='info',
                           layout=widgets.Layout(width='50%'),
                           disabled=True)


# ──────────── Gestori degli eventi ────────────
upload_button.on_click(upload_handler)

def on_process_click(b):
    global uploaded_file_path, tag_list
    if not uploaded_file_path:
        status_text.value = "<div style='color:red; padding:5px;'>Nessun file caricato!</div>"
        return

    background = text1.value
    style = text2.value
    selected_categories = [box.description for box in checkboxes if box.value]
    # Mostra messaggio di elaborazione
    status_text.value = "<div style='color:orange; padding:5px;'>Elaborazione in corso...</div>"

    try:
        # Esegui l'elaborazione
        tag_list = elaborate(uploaded_file_path, background, style, selected_categories)

        # Abilita il pulsante play dopo l'elaborazione
        play_button.disabled = False
        status_text.value = "<div style='color:green; padding:5px;'>Elaborazione completata! Pronto per la riproduzione</div>"

    except Exception as e:
        status_text.value = f"<div style='color:red; padding:5px;'>Errore durante l'elaborazione: {str(e)}</div>"
        play_button.disabled = True

def on_play_click(b):
    global uploaded_file_path
    if not uploaded_file_path:
        status_text.value = "<div style='color:red; padding:5px;'>Nessun file disponibile per la riproduzione!</div>"
        return

    status_text.value = "<div style='color:blue; padding:5px;'>Riproduzione in corso...</div>"
    try:
        generate_and_play_audio_with_images(uploaded_file_path, tag_list)
        status_text.value = "<div style='color:green; padding:5px;'>Riproduzione completata!</div>"
    except Exception as e:
        status_text.value = f"<div style='color:red; padding:5px;'>Errore durante la riproduzione: {str(e)}</div>"

process_button.on_click(on_process_click)
play_button.on_click(on_play_click)

# ──────────── Layout finale ────────────
final_ui = widgets.VBox([
    widgets.HTML("<h2 style='color:#4CAF50;'>🎼 SonART - Audio to Image Generator</h2>"),
    widgets.VBox([text1, text2]),
    widgets.HTML("<b>🎚️ Seleziona categorie audio:</b>"),
    checkbox_box,
    widgets.HTML("<b>📂 Carica un file audio:</b>"),
    upload_button,
    file_info,
    process_button,
    play_button,
    status_text
], layout=widgets.Layout(padding='20px', border='solid 1px #ccc', border_radius='10px'))

display(final_ui)

Saving Demo_Animali.mp3 to Demo_Animali (1).mp3
Numero di segmenti: 288, Forma di ogni segmento: (64000,)
Cattle, bovinae: 0.837
0:00:00


  0%|          | 0/5 [00:00<?, ?it/s]

Livestock, farm animals, working animals: 0.729
0:00:01.700000


  0%|          | 0/5 [00:00<?, ?it/s]

Dog: 0.407
0:00:02


...

  0%|          | 0/5 [00:00<?, ?it/s]

Rain: 0.542
0:00:08.700000


  0%|          | 0/5 [00:00<?, ?it/s]

Wind: 0.417
0:00:21.200000


  0%|          | 0/5 [00:00<?, ?it/s]